# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### MiniProject 4: Finance Portfolio

## Learning Objectives

At the end of the experiment, you will be able to:

* build a finance portfolio
* optimize and find the maximum return, minimum risk of a portfolio
* cluster the asset parameters to group the similar assets
* select the optimal portfolio of diversified assets


## Dataset

Dataset chosen for this mini project is Dow Jones Industrial Average (DJIA) Index viz., Apple (AAPL), American Express (AXP), Boeing (BA), Caterpillar (CAT), Cisco Systems (CSCO), Chevron (CVX), Walt Disney (DIS), Goldman Sachs (GS), The Home Depot (HD), IBM (IBM), Intel (INTC), Johnson & Johnson (JNJ), JP Morgan Chase (JPM), Coca-Cola (KO), McDonald's (MCD), 3M(MMM), Merck & Co (MRK), Microsoft (MSFT), Nike (NKE), Pfizer (PFE), Procter & Gamble (PG), Travelers (TRV), United Health Group (UNH), United Technologies (UTX), Verizon (V), Verizon (VZ), Walgreens Boots Alliance (WBA), Walmart (WMT), Exxon Mobil (XOM).

The data set considered is from April 11, 2014 to April 11, 2019. Fig. 3.4 illustrates a snapshot of the DJIA dataset.

![img](https://github.com/PaiViji/PythonFinance-PortfolioOptimization/raw/2bb0ae9f32f44ff66302ea33f53c02e746583a05/Lesson3_HeuristicPortfolioSelection/Lesson3Fig3_4.png)



## Information



The universe of stocks can truly baffle investors who wish to make the best selection of stocks for their portfolios. It is a daunting task to make a prudent selection of stocks, given the vastness of the choices and the diverse behavioural characteristics of each of these stocks with respect to itself and to one another

**Portfolio Return and Risk:**
Harry Markowtiz proposed a framework known as Mean-Variance Analysis which won him the 1990 Nobel Prize in Economics. The framework laid the foundation for what is known as the Modern Portfolio Theory, that views the *portfolio return* as the capital gain that can be expected by holding a portfolio and *portfolio risk* as the extent of capital losses that it can suffer possibly due to adverse market movements. Portfolio return and risk were modelled using the mean and variance of the portfolio's fluctuations respectively.

**Diversification Index:**
A Diversification Index quantifies diversification. There are several diversification indices discussed in the literature. Diversification Ratio proposed and patented by Yves Choueifaty in 2008 [CHO 08, CHO 13], is a diversification index of recent origin, built on the inter-dependence between assets of a portfolio. Diversification Ratio is the ratio of the weighted sum of individual asset volatilities to the portfolio's volatility.

**Efficient Frontier:** An efficient frontier is a set of investment portfolios that are expected to provide the highest returns at a given level of risk. A portfolio is said to be efficient if there is no other portfolio that offers higher returns for a lower or equal amount of risk. Where portfolios are located on the efficient frontier depends on the investor’s degree of risk tolerance.

**Sharpe Ratio:** The Sharpe ratio was developed by Nobel laureate William F. Sharpe and is used to help investors understand the return of an investment compared to its risk. The ratio is the average return earned in excess of the risk-free rate per unit of volatility or total risk. Volatility is a measure of the price fluctuations of an asset or portfolio.

## Objectives:

* How can the investor decide on which combination of assets is the best?
* How can diversification be ensured, when the assets belong to different sectors and therefore behave differently under varying market conditions?
* How would an investor determine the optimal weights, which will ensure maximum return risk for the portfolio invested in ?
* How would an investor know how much to invest in each one of the assets in the portfolio ?

In [ ]:
#@title Download the dataset
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/DJIA_Apr112014_Apr112019.csv

### Import required Packages

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

### Data Preparation

In [ ]:
#input stock dataset
StockFileName = 'DJIA_Apr112014_Apr112019.csv'

#read stock prices
df = pd.read_csv(StockFileName)
print(df.shape)
df.head()

In [ ]:
# below subset is selected based on k-means
#df = df[['BA','IBM','CAT','AAPL']]

#### Data Summarization

* Remove Date column and summarize the data

In [ ]:
Rows = len(df)
Columns = len(df.columns[1:]) # excluding date

# extract asset labels
assetLabels = df.columns[1:Columns+1].tolist()
print('portfolio asset labels:\n', assetLabels)

# extract stock prices excluding header and trading dates
dfStockPrices = df.iloc[:, 1:]

# store stock prices as an array
arStockPrices = np.asarray(dfStockPrices)
rows, cols= arStockPrices.shape

#### Compute the stock returns

* Compute the mean of returns and covariance of returns

In [ ]:
# function to compute stock returns
def StockReturnsComputing(StockPrice, Rows, Columns):
    StockReturn = np.zeros([Rows-1, Columns])
    for j in range(Columns):  # j: Assets
        for i in range(Rows-1):     #i: Daily Prices
            StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])
    return StockReturn

# compute asset returns
arReturns = StockReturnsComputing(arStockPrices, Rows, Columns)

# Alternate code
#arReturns = pd.DataFrame(arStockPrices).pct_change().dropna().values
print('k-portfolio 1 returns:\n', arReturns)

In [ ]:
#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
print('Mean returns of k-portfolio 1:\n', meanReturns)
covReturns = np.cov(arReturns, rowvar=False)
print('\nVariance-Covariance matrix of returns of k-portfolio 1: ')
print('Size  ', covReturns.shape)

### Portfolio Return and Portfolio Risk

* Apply equal weighted portfolio construction method to portfolio
* Obtain the annualized risk (%) and the expected annual return (%) of the portfolio



Let P be a portfolio comprising assets $A_1, A_2,...A_N$ with weights $W_1, W_2,...W_N$ and $r_1, r_2, ...r_N$ as the asset returns. The portfolio return r determined by a weighted summation of its individual asset returns is given by,
 $$r = W_1 . r_1 + W_2 . r_2 + .. W_N . r_N = \sum_{i=1}^N W_i . r_i $$

The annualized portfolio return assuming that the number of trading days in a year were $T_y$ ( for example, 261 out of 365 days in a year), is given by,

$$r_{Ann} = \Bigg( \sum_{i=1}^N W_i . r_i \Bigg) . T_y$$

Portfolio risk is the standard deviation of its returns and is given by,

$$ \sigma = \sqrt{\sum_i \sum_j W_i . W_j . \sigma_{ij}}$$


where $\sigma_{ij}$ is the covariance of returns between assets i and j of the portfolio, also referred to as the variance-covariance matrix of returns. Standard deviation is a prominent measure of dispersion of data borrowed from Statistics and describes the deviation or spread of the data from its mean. In the case of portfolios, standard deviation of its returns, which is its risk, helps to measure the consistency of the returns. The annualized portfolio risk in percentage,  $\sigma_{Ann}$ is given by,

$$ \sigma_{Ann} = \sqrt{ \Bigg(\sum_i \sum_j W_i . W_j . \sigma_{ij} \Bigg) . T_y}$$

where $T_y$ is the number of trading days in a year.



In [ ]:
#equal weighted portfolio construction: Annualized risk and
#expected annualized portfolio return
#trading days = 251
PortfolioSize = Columns
EqualWeightVector = np.ones((1,PortfolioSize))*(1.0/PortfolioSize)
EqWgtPortfolioRisk = np.sqrt(np.matmul((np.matmul(EqualWeightVector,covReturns)), \
                     np.transpose(EqualWeightVector)))
EqWgtAnnPortfolioRisk = EqWgtPortfolioRisk*np.sqrt(251)*100
#print(EqualWeightVector.shape, np.transpose(meanReturns).shape)
EqWgtPortfolioReturn = np.matmul(EqualWeightVector, np.transpose(meanReturns))
EqWgtAnnPortfolioReturn = 251*EqWgtPortfolioReturn * 100

print("Annualized Portfolio Risk :  %4.2f" % EqWgtAnnPortfolioRisk, "%")
print("\nAnnualized Expected Portfolio Return:  %4.2f" %  EqWgtAnnPortfolioReturn,"%")

### Cluster the Assets using K-Means

* Prepare the data for K-Means by concatenating mean returns and covariance returns

* Find the suitable k parameter and fit the model

* Label the nearest assets in clusters

In [ ]:
#prepare asset parameters for k-means clustering
#reshape for concatenation
meanReturns1 = meanReturns.reshape(len(meanReturns),1)
assetParameters = np.concatenate([meanReturns1, covReturns], axis = 1)
print('Size of the asset parameters for clustering:\n', assetParameters.shape)

In [ ]:
assetParameters

In [ ]:
#kmeans clustering of assets using the characteristic vector of
#mean return and variance-covariance vector of returns

assetsCluster= KMeans(algorithm='auto',  max_iter=600, n_clusters=4)
print('Clustering of assets completed!')
assetsCluster.fit(assetParameters)
centroids = assetsCluster.cluster_centers_
labels = assetsCluster.labels_

print('Centroids:\n', centroids)
print('Labels:\n', labels)

In [ ]:
#fixing asset labels to cluster points
print('Stocks in each of the clusters:\n',)
assets = np.array(assetLabels)
for i in range(len(set(labels))):
    print('Cluster', i+1)
    clt  = np.where(labels == i)
    assetsCluster = assets[clt]
    print(assetsCluster)

In [ ]:
for i,j in zip(meanReturns1,df.columns[1:]):
  print(i,j)

### Diversification Index

* Compute the Diversification Ratio of the Equal Weighted Portfolio

Let N be the number of assets in the portfolio spanning different asset classes or belonging to a specific class. Let $(\bar{w}=(w_1,w_2,...w_N) )$ be the weights or the proportion of capital to be invested in individual assets in the portfolio and $\bar{w}'$ its transpose. Let $(\bar{\sigma}=(\sigma_1,\sigma_2,...\sigma_N))$ be the standard deviations of returns on the assets and V, the variance-covariance matrix of returns on the assets. The Diversification Ratio of a portfolio is given as follows:

$$ \Bigg( \frac{\bar\sigma . \bar w'}{\sqrt{\bar w . V . \bar w'}}\Bigg)$$



In [ ]:
# Equal weighted portfolio: Diversification Ratio
EqWgtPortfolioAssetStdDev = np.sqrt(np.diagonal(covReturns))
EqWgtPortfolioDivRatio = np.sum(np.multiply(EqWgtPortfolioAssetStdDev, EqualWeightVector)) / EqWgtPortfolioRisk
print("Equal Weighted Portfolio:Diversification Ratio  %4.2f" % EqWgtPortfolioDivRatio)

**Summary:**

* Equal Weighted Portfolio of 29 assests:Diversification Ratio  1.57 approx

* Equal Weighted Portfolio for 4 assets:Diversification Ratio  1.32

Diversification changes if assets changes


### Inverse Volatility Weighted Portfolio

* Apply inverse volatility weighted portfolio construction method for the DJIA portfolio
* Compute Inverse volatility weighted portfolio annual risk
* Compute Inverse volatility weighted portfolio annual return
* Compute the Diversification Ratio of the Inverse Volatility Weighted Portfolio

An inverse volatility weighted portfolio is one in which highly volatile assets are allotted smaller weights and low volatile assets are allotted larger weights. Thus, the weights of the individual assets in the portfolio are proportional to the reciprocals of their individual volatilities.
The weights are given by,

$$W_i = \frac{\frac{1}{\sigma_i}}{\sum_j \frac{1}{\sigma_j}} $$

In [ ]:
#Inverse volatility weighted portfolio construction: Annualized risk and
#Expected annualized portfolio return
#Trading days = 251
InvVolWeightAssets_Risk = np.sqrt(np.diagonal(covReturns))
InvVolWeightAssets_ReciprocalRisk = 1.0/InvVolWeightAssets_Risk
InvVolWeightAssets_ReciprocalRisk_Sum = np.sum(InvVolWeightAssets_ReciprocalRisk)
InvVolWeightAssets_Weights = InvVolWeightAssets_ReciprocalRisk / InvVolWeightAssets_ReciprocalRisk_Sum
InvVolWeightPortfolio_Risk = np.sqrt(np.matmul((np.matmul(InvVolWeightAssets_Weights,
                             covReturns)), np.transpose(InvVolWeightAssets_Weights)))

#annualized risk and return
InvVolWeightPortfolio_AnnRisk = np.sqrt(251)* InvVolWeightPortfolio_Risk *100
InvVolWeightPortfolio_AnnReturn = 251* np.matmul(InvVolWeightAssets_Weights,np.transpose(meanReturns)) *100

print("Annualized Portfolio Risk: %4.2f" % InvVolWeightPortfolio_AnnRisk,"%\n")
print("Annualized Expected Portfolio Return: %4.2f" % InvVolWeightPortfolio_AnnReturn,"%")

The Diversification Ratio of the Inverse Volatility Weighted Portfolio is shown in the output.

In [ ]:
# Inverse volatility weighted portfolio: Diversification Ratio
InvVolWeightAssets_Risk= np.sqrt(np.diagonal(covReturns))
InvVolWeightPortfolioDivRatio = np.sum(np.multiply(InvVolWeightAssets_Risk,
                                                   InvVolWeightAssets_Weights)) / InvVolWeightPortfolio_Risk
print("\n Inverse Volatility  Weighted Portfolio:Diversification Ratio %4.2f"
      % InvVolWeightPortfolioDivRatio)

### Efficient Frontier
**Optimize the portfolio weights:**

* Initialize the random weights for each portfolio
* Calculate return, volatility for each portfolio


The efficient set obtained by the Mean-Variance Optimization model can be graphically represented by what is called an efficient frontier. An efficient frontier is a risk-return tradeoff graph, which describes a set of optimal portfolios that yield the highest expected portfolio return for a defined level of risk or the lowest possible risk for a defined level of expected portfolio return. It graphs the optimal structure of the portfolio which yields the maximum expected return for a given level of risk or vice-versa.


In [ ]:
# Define an empty list for portfolio returns, volatility and asset weights
p_ret = []
p_vol = []
p_weights = []

num_assets = Columns
print("Number of assets: ", num_assets)
num_portfolios = 10000
print("Number of portfolios: ", num_portfolios)

In [ ]:
covReturns[:5,:5].shape

In [ ]:
np.random.seed(1)
for portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    # sum of weights must be 1
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    # Returns are the product of individual expected returns of asset and its weights
    returns = np.dot(weights, meanReturns)
    p_ret.append(returns)
    # Portfolio Variance
    volat = np.sqrt(np.matmul((np.matmul(weights,covReturns)), np.transpose(weights)))
    ann_volat = volat*np.sqrt(251)*100
    # var = covReturns.mul(weights, axis=0).mul(weights, axis=1).sum().sum()
    # # Daily standard deviation
    # sd = np.sqrt(var)
    # # Annual standard deviation = volatility
    # ann_sd = sd * np.sqrt(250)
    p_vol.append(ann_volat)

In [ ]:
df.columns

In [ ]:
df.columns.tolist()[1:6]

In [ ]:
data_ = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(df.columns.tolist()[1:6]):
    print(counter, symbol)
    data_[symbol + '_weight'] = [w[counter] for w in p_weights]

# Create dataframe of the 10000 portfolios
portfolios  = pd.DataFrame(data_)

In [ ]:
portfolios.head()

#### Visualize Efficient Frontier

* Visualize the volatility and returns of 1000 portfolios
* Find the minimum variance portfolio and plot the weights of assets

In [ ]:
# Visualize efficient frontier
plt.figure(figsize=(8,6))
plt.scatter(portfolios['Volatility'], portfolios['Returns'], marker='o', s=10, alpha=0.3)
plt.title("Efficient Frontier")
plt.xlabel("Risk")
plt.ylabel("Returns")
plt.grid()
plt.show()

In [ ]:
# Minimum variance portfolio
min_var_port = portfolios.iloc[np.argmin(portfolios['Volatility'])]
min_var_port

In [ ]:
# Weights for Minimum variance portfolio
plt.figure(figsize=(19,7))
sns.barplot(x = min_var_port[2:].index, y = min_var_port[2:])
plt.ylabel("Weights")
plt.show()

In [ ]:
# Visualize Efficient frontier
plt.subplots(figsize=[8,6])
plt.scatter(portfolios['Volatility'], portfolios['Returns'], marker='o', s=10, alpha=0.3)

# Visualize Minimum variance portfolio
plt.scatter(min_var_port[1], min_var_port[0], color='r', marker='*', s=300)
plt.title("Efficient Frontier")
plt.xlabel("Risk")
plt.ylabel("Returns")
plt.grid()
plt.show()

### Sharpe Ratio

* Calculate Sharpe Ratio for the assets

Sharpe Ratio, developed by Nobel Laureate William F Sharpe, is a measure of calculating risk adjusted return. It serves to help investors know about the returns on their investments relative to the risks they hold. The Sharpe Ratio is defined as

$$Sharpe Ratio = \frac{R_P - R_f}{\sigma_p} $$

where:

$R_p$ = return of portfolio

$R_f$ = risk-free rate

$\sigma_p$ = standard deviation of the portfolio’s excess return
​

**Note:** The risk-free rate of return is the return of an investment with zero risks, meaning it's the return investors could expect for taking no risk.

The annual average risk free rate of return in USA during April 2019 was 3%. The daily risk free rate is computed as

$$R_f^{Daily} = \Big(1 + R_f^{Yearly} \Big)^{\frac{1}{360}} - 1 $$



In [ ]:
# Risk-free return
rf = 0.00060
# Sharpe ratios for different weight combinations
sharpe_ratios = (portfolios['Returns'] - rf)/portfolios['Volatility']

# Highest sharpe ratio
optimal_idx = np.argmax(sharpe_ratios)
print("Highest Sharpe ratio: ", sharpe_ratios[optimal_idx])

In [ ]:
# Tangent portfolio
optimal_risky_port = portfolios.iloc[optimal_idx]
optimal_risky_port

In [ ]:
# Weights for Tangency portfolio
plt.figure(figsize=(19,7))
sns.barplot(x = optimal_risky_port[2:].index, y = optimal_risky_port[2:])
plt.ylabel("Weights")
plt.show()

In [ ]:
# Diference b/w tangent portfolio and minimum variance portfolio
optimal_risky_port - min_var_port

#### Plot the portfolio

* Find and plot the optimal portfolio using Sharpe Ratio
* Highlight the minimum risk and maximum return of the portfolio

In [ ]:
# Visualize Efficient frontier
plt.subplots(figsize=[8,6])
plt.scatter(portfolios['Volatility'], portfolios['Returns'], marker='o', s=10, alpha=0.3)

# Visualize Minimum variance portfolio
plt.scatter(min_var_port[1], min_var_port[0], color='r', marker='*', s=300, label="min risk")

# Visualize optimal or tangent portfolio
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=300,label="max return")
plt.title("Efficient Frontier")
plt.xlabel("Risk")
plt.ylabel("Returns")
plt.legend()
plt.grid()
plt.show()

### Report Analysis

* Try with a subset of assets and comment on diversification ratio

* Comment on the portfolio that consists of assets with highest return (irrespective of risk)

* Discuss the optimized portfolio and weights of the assets

* Based on K-Means clusters, find the subset of assets which is exhibiting greater sharpe ratio
